In [1]:
import scsims
import pandas as pd 
import anndata as an
from scsims.autoencoder import *
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import numpy as np
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, Subset, DataLoader

In [2]:
traindata = an.read_h5ad('../data/bhaduri/primary_RAWCOUNTS_T.h5ad')
testdata = an.read_h5ad('../data/benchmark/human_lognorm.h5ad')

In [3]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    return Subset(dataset, train_idx), Subset(dataset, val_idx)

train, val = train_val_dataset(AEDataset(traindata.X))

test = AEDataset(testdata.X)

In [9]:
train_compresser = AutoEncoder(
    data_shape=traindata.shape[1],
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.001,
        'weight_decay': 0.001,
    }
)

test_compresser = AutoEncoder(
    data_shape=testdata.shape[1],
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.001,
        'weight_decay': 0.001,
    }

)

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/scsims/autoencoder.py:16: UserWarning: Output_dim < 500, final layer has increasing dimensionality in Encoder
  warnings.warn(


In [10]:
trainloader, valloader = DataLoader(train, batch_size=4, shuffle=True), DataLoader(val, batch_size=4)

Now we can train our model

In [11]:
trainer = pl.Trainer(
    max_epochs=50,
    logger=WandbLogger(project='Train_encoder')
)

trainer.fit(train_compresser, trainloader, valloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 392 M 
1 | decoder | Decoder | 392 M 
2 | loss    | MSELoss | 0     
------------------------------------
785 M     Trainable params
0         Non-trainable params
785 M     Total params
3,140.788 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]


KeyboardInterrupt

